<a href="https://colab.research.google.com/github/purinreki/WET2.0/blob/main/331_wet3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## WET 3.0 - Global Wetland Extent Tool
### WET Water Resources, JPL Spring 2023
#### Lori Berberian (Project Lead), Kaely Harris, Mitch Porter, and Emma Waugh

#### 1: Import Modules and Initialize GEE
**INPUT REQUIRED** : Enter Google Earth Engine account credentials when prompted in a new window

In [22]:
# Import necessary libraries

#!pip install matplotlib.pyplot
#!pip install tkcalendar
#!pip install tkinter
#!pip install rasterio
#!pip install pandas
#!pip install geopandas
!pip install oauth2client

#import geemap
#import seaborn
#import matplotlib.pyplot as plt
#import pandas as pd
#import tkinter as tk
#from tkcalendar import DateEntry
#import rasterio
#import geopandas as gpd
#import oauth2client

# Trigger the authentication flow.
#ee.Authenticate()
NASA_SWC_GEE = 'ee-trejor1823'
# Initialize the library.
ee.Initialize(project=NASA_SWC_GEE)

#### 2: Select Date Range & ROI
**INPUT REQUIRED** : Draw ROI on map
- rerun the following cell to reset map if needed

In [23]:
# Initialize map
Map = geemap.Map()

Map.addLayerControl()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

##### Draw ROI on above map, then run the following cell

In [ ]:
# Save ROI as FeatureCollection
roi = ee.FeatureCollection(Map.draw_features)

# Add the ROI as a layer to the map
Map.addLayer(roi, {'color': 'grey'}, 'ROI')

#### 3: Optional: Snow Cover Detection

##### Import snow cover imagery and visualize timeseries

In [ ]:
# Adjust dates to desired date range
snow_start_date = '2020-10-15'
snow_end_date = '2021-10-25'

# Create MODIS image collection from defined date range and ROI
modis = ee.ImageCollection('MODIS/006/MOD10A1').filter(ee.Filter.date(snow_start_date, snow_end_date))

# Select Snow Cover product
snowCover = modis.select('NDSI_Snow_Cover')

# Define a function to clip each image in the collection to the ROI
# Get a list of clipped images from the collection
def clip_image(image):
    return image.clip(roi)
clipped_collection = snowCover.map(clip_image)
clipped_images = clipped_collection.toList(clipped_collection.size())

# Compute the mean value of the snow cover pixels within the ROI
# This function uses the reduceRegion function to apply a reducer (ee.Reducer.mean()) over the
# specified geometry (ROI) and scale (500 meters).
def roi_mean(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=500).get('NDSI_Snow_Cover')
    return img.set('date', img.date().format()).set('mean',mean)

# Creates a new collection with the mean values of snow cover for each image.
roi_reduced_imgs = modis.map(roi_mean)

# Create a nested list of the date and mean snow cover values for each image in the collection.
# Convert to dataframe
nested_list = roi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','mean']).values().get(0)

# Set date column to index of the dataframe
df = pd.DataFrame(nested_list.getInfo(), columns=['date','mean'])
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

# Create figure and axes, with the dimensions set to 15 x 7.
fig, ax = plt.subplots(figsize=(15,7))

# Create line plot of the daily mean snow cover values over time
sns.lineplot(data=df, ax=ax)
ax.set_ylabel('Snow Cover',fontsize=20)
ax.set_xlabel('Date',fontsize=20)
ax.set_title('Daily Mean Snow Cover',fontsize=20);
ax.set_ylim([0, 100])
ax.set_yticks(range(0, 101, 10))

# A green shading below y values of 10:
ax.fill_between(df.index, 0, df['mean'], where=(df['mean']<10), color='green', alpha=0.3)

# Create figure and axes, with the dimensions set to 15 x 7.
fig, ax = plt.subplots(figsize=(15,7))

# Compute 7-day rolling average of daily mean snow cover and plot
window=7
sns.lineplot(data=df.rolling(window).mean(), ax=ax)
ax.set_ylabel('Snow Cover',fontsize=20)
ax.set_xlabel('Date',fontsize=20)
ax.set_title(f'Daily Mean Snow Cover ({window} day rolling avg.)',fontsize=20)
ax.set_ylim([0, 100])
ax.set_yticks(range(0, 101, 10))

# Add a horizontal line at y=10 with green shading below
ax.axhline(y=10, color='green', linestyle='--', alpha=0.5)
ax.fill_between(df.index, 0, 10, color='green', alpha=0.3)

# Creates a plot that shows the daily mean snow cover values over time, with a rolling average
# of 7 days and a horizontal line at y=10, indicating a threshold for significant snow cover.
# The green shading below the line y=10 highlights periods when the snow cover was particularly low.
df

Limitations of the snow cover product: inaccurate indication of snow cover.
When 7-day rolling average is below the y = 10 line, the snow cover can likely be ignored

#### 4: Obtain SAR Imagery for Given Date Range and ROI

In [ ]:
# Create image collection
image_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') \
                .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                .filterBounds(roi)

# Clip filtered collection to ROI
def clip_image(image):
    return image.clip(roi)
image_collection = image_collection.map(clip_image)

##### Choose date range for SAR imagery based on above snow cover timeseries
**INPUT REQUIRED** : Enter time in calendar widget

Close widget before running the next cell

In [ ]:
root = tk.Tk()

# Create a starting date picker widget
start_date_label = tk.Label(root, text = "Starting Date (YYYY-MM-DD):")
start_date_label.pack(padx = 10, pady = 10)
start_cal = DateEntry(root, width = 12, background = 'darkblue',
                      foreground = 'white', borderwidth=2, date_pattern = 'yyyy-mm-dd')
start_cal.pack(padx = 10, pady = 10)

# Create an end date picker widget
end_date_label = tk.Label(root, text = "Ending Date (YYYY-MM-DD):")
end_date_label.pack(padx = 10, pady = 10)
end_cal = DateEntry(root, width = 12, background = 'darkblue',
                    foreground = 'white', borderwidth = 2, date_pattern = 'yyyy-mm-dd')
end_cal.pack(padx = 10, pady = 10)

# Define a function to filter the image collection using the selected dates
def filter_image_collection():
    global filtered_collection
    start_date = start_cal.get_date().strftime('%Y-%m-%d')
    end_date = end_cal.get_date().strftime('%Y-%m-%d')
    filtered_collection = image_collection.filter(ee.Filter.date(start_date, end_date))
    print("Filtered Collection Size: ", filtered_collection.size().getInfo())
    print("Start Date: ", start_date)
    print("End Date: ", end_date)
    return filtered_collection

#Create a button to filter the image collection
filter_button = tk.Button(root, text = "Filter", command = filter_image_collection)
filter_button.pack(padx = 10, pady = 10)

root.mainloop()

Optional: Print list of images in collection

In [ ]:
# Uncomment the below lines to get a list of image dates from the collection
image_dates = filtered_collection.toList(filtered_collection.size())
image_dates

#### 5: Prepare Imagery for Analysis

##### Optional: Print relative orbit numbers. If there are more than one, that means the ROI covers more than one Sentinel-1 swath.
Be aware that the change in incidence angle of the intersecting swaths can affect comparisons in your data.

In [ ]:
relative_orbits = filtered_collection.distinct('relativeOrbitNumber_start').aggregate_array('relativeOrbitNumber_start')
print('Relative orbit numbers:', relative_orbits.getInfo())

##### Create VV/VH ratio band

In [ ]:
def add_ratio(image):
    return image.addBands(image.select('VV').divide(image.select('VH')).rename('VV/VH'))

filtered_collection = filtered_collection.map(add_ratio)

##### Optional: Rolling averages to despeckle

In [ ]:
# Temporal rolling average:
# Define the temporal averaging parameter
Nave_rolling = 2  # number of images in temporal rolling average

# Define a function to apply the temporal rolling average to an image
def apply_rolling_average(image):
    # Define a collection of Nave_rolling images before and after the current image
    collection = filtered_collection.filterDate(image.date().advance(-((Nave_rolling - 1) * 12), 'month'),
                                                 image.date().advance(((Nave_rolling - 1) * 12), 'month'))
    # Calculate the mean of the collection
    mean_image = collection.mean()
    return mean_image.copyProperties(image)

# Apply the temporal rolling average to the filtered collection
filtered_collection = filtered_collection.map(apply_rolling_average)

vis_params = {'bands': ['VV'], 'min': 0.0, 'max': 0.5, 'opacity': 1.0, 'gamma': 2}
Map.addLayer(filtered_collection.first(), vis_params, "Temporal Rolling Avg.")
Map

In [ ]:
# Spatial rolling average:
# Define the spatial averaging parameter
ispat = 1
if ispat == 1:
    pixel_spacing = 10  # Sentinel-1 SAR data has a pixel spacing of 10 m
    kernel_size = int(pixel_spacing / 2) + 1
    kernel = ee.Kernel.square(kernel_size, 'meters')

    # Define a function to apply the spatial averaging kernel to an image
    def apply_kernel(image):
        return image.convolve(kernel)

    # Apply the spatial averaging kernel to the SAR collection
    filtered_collection = filtered_collection.map(apply_kernel)

# Visualize the filtered collection clipped to the ROI
roi_vis_params = {'bands': ['VV'], 'min': 0.0, 'max': 0.5, 'opacity': 1.0, 'gamma': 2}
vis_params_class = {'bands': ['classified'], 'min': 1, 'max': 3, 'opacity': 1.0}
Map.addLayer(filtered_collection.first(), roi_vis_params, "VV Spatial Rolling Avg.")
Map

##### Separate ascending and descending orbits
Depending on location, the collection will have ascending or descending orbits or both.
Because of differences in incidence angle, you need to separate them into separate collections and do analysis separately.

In [ ]:
sent_asc = filtered_collection.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
sent_desc = filtered_collection.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

if sent_asc.size().getInfo() > 0 and sent_desc.size().getInfo() > 0:
    print('Both ascending and descending orbits are present.')
else:
    print('Only one orbit direction is present.')

# Print the number of images in each collection
print('Number of images in ascending collection:', sent_asc.size().getInfo())
print('Number of images in descending collection:', sent_desc.size().getInfo())

In [ ]:
def mosaic_by_date(ic):

# Extract date (without time) from image properties and store it in a new property 'date'
    ic = ic.map(lambda img: img.set('date', ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')))

# Get the distinct dates
    distinct_dates = ic.distinct('date')


# Define a function to mosaic images for each distinct date
    def mosaic_on_date(date_image):
        date = ee.Date(date_image.get('date'))
        images_on_date = ic.filterDate(date, date.advance(1, 'day'))
        return images_on_date.mosaic().copyProperties(date_image, ['date'])

# Map the function over the distinct_dates collection
    mosaic_list = distinct_dates.map(mosaic_on_date)
    return ee.ImageCollection(mosaic_list)

sent_desc = mosaic_by_date(sent_desc)
sent_asc = mosaic_by_date(sent_asc)

print('Number of mosaicked images in ascending collection:', sent_asc.size().getInfo())
print('Number of mosaicked images in descending collection:', sent_desc.size().getInfo()) #returns error

#### 6: Optional: Calculate relative change from mean for each image & display on map

In [ ]:
# compute the mean image of the collection
mean_asc = sent_asc.mean()
mean_desc = sent_desc.mean()

# iterate over each image in the collection, and calculate the relative difference from the mean
def relative_change_asc(image):
    return image.subtract(mean_asc).divide(mean_asc).abs()

def relative_change_desc(image):
    return image.subtract(mean_desc).divide(mean_desc).abs()

relchange_asc = sent_asc.map(relative_change_asc)
relchange_desc = sent_desc.map(relative_change_desc)

# set visualization parameters - adjust min and max values as needed
vis_params_relchange_vv = {'bands': ['VV'], 'min': 0.0, 'max': 0.8, 'opacity': 1.0, 'palette': ['000000', 'FFFFFF']}

# get the list of images in the collection, then iteratively plot each image in the collection
image_list_asc = relchange_desc.toList(relchange_asc.size())
for i in range(relchange_asc.size().getInfo()):
    image_index = i + 1
    image = ee.Image(image_list_asc.get(i))
    Map.addLayer(image, vis_params_relchange_vv, f'Image {image_index}')


image_list_desc = relchange_desc.toList(relchange_desc.size())
for i in range(relchange_desc.size().getInfo()):
    image_index = i + 1
    image = ee.Image(image_list_desc.get(i))
    Map.addLayer(image, vis_params_relchange_vv, f'Image {image_index}')

Map.addLayerControl()
Map

#### 7: Visualize Data
- Toggle layers to view
- Use the trash can to "clear all" geometries to get rid of the blue box on top

In [ ]:
vp_angle = {'bands': ['angle'], 'min': 0, 'max': 45, 'opacity': 1.0, 'palette': ['3cb371', '90ee90']}
vis_params_vv = {'bands': ['VV'], 'min': 0.0, 'max': 0.5, 'opacity': 1.0, 'gamma': 2}
vis_params_vh = {'bands': ['VH'], 'min': 0.0, 'max': 0.5, 'opacity': 1.0, 'gamma': 2}
vis_params_ratio = {'bands': ['VV/VH'], 'min': 0.0, 'max': 20, 'opacity': 1.0, 'gamma': 2}

Map.addLayer(sent_desc, vp_angle, "Incidence Angle")

# add ascending layers to map, if present
if sent_asc.size().getInfo() > 0:
    Map.addLayer(sent_asc.mean(), vis_params_vv, "Mean Ascending VV")
    Map.addLayer(sent_asc.mean(), vis_params_vh, "Mean Ascending VH")
    Map.addLayer(sent_asc.mean(), vis_params_ratio, "Mean Ascending VV/VH")

# add descending layers to map, if present
if sent_desc.size().getInfo() > 0:
    Map.addLayer(sent_desc.mean(), vis_params_vv, "Mean Descending VV")
    Map.addLayer(sent_desc.mean(), vis_params_vh, "Mean Descending VH")
    Map.addLayer(sent_desc.mean(), vis_params_ratio, "Mean Descending VV/VH")

Map

#### 8: Classify Imagery
**INPUT REQUIRED** : Enter threshold values from thresholding script

In [ ]:
# This code defines a function classify_image that takes a Sentinel-1 image and reclassifies
# each pixel based on threshold values for VV and VH bands, assigning each pixel to a new class
# based on whether it represents open water, inundated vegetation, or no water.
# The function creates a new band for the classified image, assigns unclassified pixels to a new class
# called "no data", and returns the input image with the new band added.
import numpy as np
import rasterio

def classify_image(image, open_water_thresh, inundated_vegetation_thresh, ratio_thresholds, no_water_thresh):
    """
    Create a new band by reclassifying each pixel in the input Sentinel-1 image
    based on the given threshold values for VV and VH bands.

    Args:
        image (ee.Image): Sentinel-1 image to classify.
        open_water_thresh (float): Threshold for open water classification.
        inundated_vegetation_thresh (float): Threshold for inundated vegetation classification.
        ratio_thresholds (list): List of two ratio thresholds for no water and inundated vegetation.
        no_water_thresh (float): Threshold for no water classification.

    Returns:
        output_image (ee.Image): Classified Sentinel-1 image.
    """

    # Select the VV and VH bands
    vv = image.select('VV')
    vh = image.select('VH')

    # Calculate VV/VH ratio
    ratio = vv.divide(vh)

    # Create a new band for the classified image
    classified = ee.Image(0).float()

    # Reclassify each pixel based on threshold values
    classified = classified.where(vv.lt(open_water_thresh), 1)  # open water
    classified = classified.where(vv.gte(open_water_thresh).And(vv.lt(inundated_vegetation_thresh)), 2)  # inundated vegetation
    classified = classified.where(vv.gte(0.1).And(vv.lt(0.2)).And(ratio.lt(ratio_thresholds[0])), 3)  # no water
    classified = classified.where(vv.gte(0.1).And(vv.lt(0.2)).And(ratio.gte(ratio_thresholds[0])), 2)  # inundated vegetation
    classified = classified.where(vv.gte(0.2).And(vv.lt(0.3)).And(ratio.lt(ratio_thresholds[1])), 3)  # no water
    classified = classified.where(vv.gte(0.2).And(vv.lt(0.3)).And(ratio.gte(ratio_thresholds[1])), 2)  # inundated vegetation
    classified = classified.where(vv.gte(no_water_thresh), 3) # no water

    # Assign unclassified pixels to new class called no data with a value of 0
    classified = classified.where(classified.eq(0), 0)

    return image.addBands(classified)

# Define threshold values
open_water_thresh = 0.01
inundated_vegetation_thresh = 0.1
# 0.1 - 0.2 VV value ratio first, 0.2 - 0.3 VV value ratio second
ratio_thresholds = [5.5, 8.5]
# anything above this VV threshold is classified as no water
no_water_thresh = 0.3

# Classify the Sentinel-1 descending imagery
classified_desc = sent_desc.map(lambda img: classify_image(img, open_water_thresh, inundated_vegetation_thresh, ratio_thresholds, no_water_thresh))

# Classify the Sentinel-1 ascending imagery
classified_asc = sent_asc.map(lambda img: classify_image(img, open_water_thresh, inundated_vegetation_thresh, ratio_thresholds, no_water_thresh))

# Defines a list of class values and a color palette for visualizing the classified image
class_values = [0, 1, 2, 3]
class_palette = ['#ffffff', '#00008b', '#ADD8E6', '#D2B48C']

# Add the ascending imagery to the map
if sent_asc.size().getInfo() > 0:

    classified_asc_list = classified_asc.toList(classified_asc.size())
    for i in range(classified_asc_list.length().getInfo()):
        img = ee.Image(classified_asc_list.get(i))
        date = ee.Date(img.get('date')).format('YYYY-MM-dd').getInfo()
        landcover = img.select('constant').set('class_values', class_values).set('class_palette', class_palette)
        landcover = landcover.clip(roi)
        Map.addLayer(landcover, {'min': 0, 'max': 3, 'palette': class_palette}, f'Ascending {date}')

# Add the descending imagery to the map
if sent_desc.size().getInfo() > 0:

    classified_desc_list = classified_desc.toList(classified_desc.size())
    for i in range(classified_desc_list.length().getInfo()):
        img = ee.Image(classified_desc_list.get(i))
        date = ee.Date(img.get('date')).format('YYYY-MM-dd').getInfo()
        landcover = img.select('constant').set('class_values', class_values).set('class_palette', class_palette)
        landcover = landcover.clip(roi)
        Map.addLayer(landcover, {'min': 0, 'max': 3, 'palette': class_palette}, f'Descending {date}')

Map

In [ ]:
def calculate_areas(image, geometry):
    area_image = image.select('constant').clip(geometry)
    fixed_scale = 10  # You can adjust this value as needed. Higher values will overestimate dominant features and underestimate less common features

    # Calculate the area for each land class separately
    areas = {}
    for land_class, land_class_name in zip([0, 1, 2, 3], ['No Data', 'Open Water', 'Inundated Vegetation', 'No Water']):
        masked_image = area_image.eq(land_class)
        area = masked_image.reduceRegion(reducer=ee.Reducer.sum(), geometry=geometry, scale=fixed_scale, maxPixels=1e9).getInfo()["constant"]
        areas[land_class_name] = (area * fixed_scale**2) / 1e6

    return areas

# Calculate the area of each land cover type for each image in the image collection
if sent_desc.size().getInfo() > 0:
    areas_data_desc = []
    for i in range(classified_desc_list.length().getInfo()):
        image = ee.Image(classified_desc_list.get(i))
        areas = calculate_areas(image, roi)
        areas['date'] = ee.Date(image.get('date')).format('YYYY-MM-dd').getInfo()
        areas_data_desc.append(areas)

    # Convert the list of area dictionaries to a pandas DataFrame
    areas_df_desc = pd.DataFrame(areas_data_desc)
    areas_df_desc['date'] = pd.to_datetime(areas_df_desc['date'])
    areas_df = areas_df_desc.set_index('date')

    # Print the content of the DataFrame
    areas_df_desc = areas_df_desc.sort_index()
    print("DataFrame content for descending imagery:")
    print(areas_df)

    # Print basic statistics of the DataFrame
    print("\nBasic statistics for descending imagery:")
    print(areas_df.describe())

    # Create a bar plot for the land cover types
    bar_colors = ['#D3D3D3', '#00008b', '#ADD8E6', '#D2B48C']
    ax = areas_df.plot(kind='bar', stacked=False, figsize=(12, 6), color=bar_colors)
    ax.set_ylabel('Area (km²)')
    ax.set_xlabel('Date')
    ax.set_title('Total Area by Land Cover Type Across Image Collection for Descending Imagery')

    plt.xticks(range(len(areas_df.index)), [x.strftime('%Y-%m-%d') for x in areas_df.index], rotation=45)
    plt.legend(['No data', 'Open water', 'Inundated vegetation', 'No water'], loc='upper left', bbox_to_anchor=(1.02, 1.0))
    plt.subplots_adjust(right=0.8)
    plt.show()

# Calculate the area of each land cover type for each image in the image collection
if sent_desc.size().getInfo() > 0:
    areas_data_asc = []
    for i in range(classified_asc_list.length().getInfo()):
        image = ee.Image(classified_asc_list.get(i))
        areas = calculate_areas(image, roi)
        areas['date'] = ee.Date(image.get('date')).format('YYYY-MM-dd').getInfo()
        areas_data_asc.append(areas)

    # Convert the list of area dictionaries to a pandas DataFrame
    areas_df_asc = pd.DataFrame(areas_data_asc)
    areas_df_asc['date'] = pd.to_datetime(areas_df_asc['date'])
    areas_df_asc = areas_df_asc.set_index('date')

    # Print the content of the DataFrame
    areas_df_asc = areas_df_asc.sort_index()
    print("DataFrame content for ascending imagery:")
    print(areas_df_asc)

    # Print basic statistics of the DataFrame
    print("\nBasic statistics for ascending imagery:")
    print(areas_df_asc.describe())

    # Create a bar plot for the land cover types for ascending imagery
    bar_colors = ['#D3D3D3', '#00008b', '#ADD8E6', '#D2B48C']
    ax = areas_df_asc.plot(kind='bar', stacked=False, figsize=(12, 6), color=bar_colors)
    ax.set_ylabel('Area (km²)')
    ax.set_xlabel('Date')
    ax.set_title('Total Area by Land Cover Type Across Image Collection for Ascending Imagery')

    plt.xticks(range(len(areas_df_asc.index)), [x.strftime('%Y-%m-%d') for x in areas_df_asc.index], rotation=45)
    plt.legend(['No data', 'Open water', 'Inundated vegetation', 'No water'], loc='upper left', bbox_to_anchor=(1.02, 1.0))
    plt.subplots_adjust(right=0.8)
    plt.show()
